In [1]:
# Get Dataset
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import levene, ttest_ind, wilcoxon, mannwhitneyu
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, roc_auc_score, accuracy_score
from scipy.stats import shapiro
import xgboost as xgb

In [2]:
data = pd.read_csv('./DataExtract.csv')

In [3]:
data.head()

,pdc,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,...,brand_cost,ratio_G_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_G_total_cost_post,pdc_80_flag,drug_class,patient_key
0,0.894444,0,0,8,0,17,3,6.966667,1.000000,0,...,239.699550,0.710850,17,0,4332.043215,0.000000,1.000000,1,*ANTICOAGULANTS*,121
1,0.333333,0,0,4,0,15,5,14.466667,0.101382,0,...,2984.927229,0.010155,2,13,196.359216,3001.501507,0.061403,0,*ANTIDIABETICS*,168
2,0.077778,0,0,32,0,11,7,3.833333,0.739130,0,...,0.000000,1.000000,10,1,203.603270,1154.818505,0.149882,0,*ANTICOAGULANTS*,267
3,0.333333,0,0,57,0,16,6,24.000000,0.958333,0,...,160.804799,0.291008,15,1,140.952640,265.057382,0.347165,0,*ANTICOAGULANTS*,468
4,0.866667,0,0,5,0,16,4,18.000000,0.888889,0,...,0.000000,1.000000,14,2,671.755173,735.661568,0.477297,1,*ANTIDIABETICS*,499


In [4]:
data.patient_key.nunique()

3550

In [5]:
data.shape #no duplicate records

(3550, 95)

In [6]:
data_pdc_pop = data.groupby(['drug_class', 'pdc_80_flag']).patient_key.nunique()

In [7]:
data_pdc_pop

drug_class        pdc_80_flag
*ANTICOAGULANTS*  0              1437
                  1               391
*ANTIDIABETICS*   0              1051
                  1               671
Name: patient_key, dtype: int64

In [15]:
# Filter data for ANTIDIABETICS drug class
AD = data[data['drug_class'].str.contains('ANTIDIABETICS')]
AD = AD.rename(columns=lambda x: x.lower())


In [16]:
AD['pdc_80_flag'].value_counts()

0    1051
1     671
Name: pdc_80_flag, dtype: int64

In [17]:
AD_v2 = AD.drop(columns = ['pdc_80_flag',  'drug_class', 'patient_key', 'pdc_cat', 'pdc'])

### model 2: feature selection using Logistic Lasso Regression

In [18]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(AD_v2, y, test_size=0.2, random_state=42)

In [19]:
cat = ['age_grpn', 'regionn', 'sexn']
X_train_con2 = X_train2.drop(columns = cat).fillna(0)
X_train_cat2 = X_train2[cat]
X_test_con2 = X_test2.drop(columns = cat).fillna(0)
X_test_cat2 = X_test2[cat]

In [20]:
scaler = StandardScaler()
X_train_normalized2 = scaler.fit_transform(X_train_con2)
X_test_normalized2 = scaler.transform(X_test_con2)

In [21]:
X_train_normalized_df2 = pd.DataFrame(X_train_normalized2, columns = X_train_con2.columns)
X_test_normalized_df2 = pd.DataFrame(X_test_normalized2, columns = X_train_con2.columns)
X_train_normalized_df2.reset_index(drop=True, inplace=True)
X_train_cat2.reset_index(drop=True, inplace=True)
X_test_normalized_df2.reset_index(drop=True, inplace=True)
X_test_cat2.reset_index(drop=True, inplace=True)

In [22]:
X_train_normalized_df2.tail(3)

,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,post_er_flag,...,numofgen,numofbrand,generic_cost,brand_cost,ratio_g_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_g_total_cost_post
1374,-0.269487,-0.178651,-0.374638,-0.329215,-0.833134,-0.730952,-0.330888,0.990640,-0.290262,-0.395185,...,-0.018001,-0.682009,0.277723,-0.367234,1.110007,-0.568545,-0.861408,-0.446864,-0.487602,1.364182
1375,-0.269487,-0.178651,2.081387,1.244845,2.738248,4.732521,5.281420,-0.147297,-0.290262,2.530460,...,2.363316,1.517351,1.353713,2.372928,-1.070221,2.584154,1.507994,2.708295,2.921986,-0.796610
1376,-0.269487,-0.178651,-0.486276,-0.329215,-0.021456,-0.482612,0.232716,0.990640,-0.290262,-0.395185,...,-0.018001,-0.462073,0.014375,-0.359757,0.939221,0.448455,-0.861408,0.494360,-0.487602,1.364182


In [23]:
X_train_normalized_df2.shape, X_train_cat2.shape, y_train2.shape

((1377, 87), (1377, 3), (1377,))

In [24]:
X_train_lasso = pd.concat([X_train_normalized_df2, X_train_cat2], axis=1)
X_test_lasso = pd.concat([X_test_normalized_df2, X_test_cat2], axis=1)

In [25]:
X_train_lasso.tail(3)

,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,post_er_flag,...,brand_cost,ratio_g_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_g_total_cost_post,age_grpn,regionn,sexn
1374,-0.269487,-0.178651,-0.374638,-0.329215,-0.833134,-0.730952,-0.330888,0.990640,-0.290262,-0.395185,...,-0.367234,1.110007,-0.568545,-0.861408,-0.446864,-0.487602,1.364182,3,1,1
1375,-0.269487,-0.178651,2.081387,1.244845,2.738248,4.732521,5.281420,-0.147297,-0.290262,2.530460,...,2.372928,-1.070221,2.584154,1.507994,2.708295,2.921986,-0.796610,3,4,2
1376,-0.269487,-0.178651,-0.486276,-0.329215,-0.021456,-0.482612,0.232716,0.990640,-0.290262,-0.395185,...,-0.359757,0.939221,0.448455,-0.861408,0.494360,-0.487602,1.364182,3,2,2


In [26]:
y_train2.head()

3358    0
2018    1
2911    0
1418    0
1891    1
Name: pdc_80_flag, dtype: int64

In [27]:
lasso_log_cv = LogisticRegressionCV(
    Cs=10,                   # Range of C values (inverse of regularization strength)
    cv=5,              
    penalty='l1',             # L1 regularization (Lasso)
    solver='saga',            # saga solver supports L1 regularization
    scoring='roc_auc',             # You can use other metrics like 'accuracy', 'roc_auc', etc.
    max_iter=10000,           # Higher iteration count for convergence
    class_weight='balanced'   #algorithm gives more weight to the minority class during training.

)


In [28]:
lasso_log_cv.fit(X_train_lasso, y_train2)

LogisticRegressionCV(Cs=10, class_weight='balanced', cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=10000,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring='roc_auc', solver='saga', tol=0.0001,
           verbose=0)

In [29]:
best_c = lasso_log_cv.C_[0]
print(best_c)

0.046415888336127774


In [30]:
coefficients = lasso_log_cv.coef_[0] 
# Find the indices of non-zero coefficients
non_zero_indices = np.where(coefficients != 0)[0]
feature_names = AD_v2.columns
non_zero_feature_names = feature_names[non_zero_indices]

In [31]:
print(non_zero_feature_names) 

Index(['num_gpi6_post', 'adjust_total_30d_post', 'generic_rate_post',
       'post_ip_flag', 'post_er_flag', 'post_rx_cost', 'post_total_cost',
       'regionn', 'idx_prodtypen', 'chronic_pain_fibro', 'copd', 'hepatitis',
       'hiv_aids', 'hypertension', 'smoking', 'pre_ip_cost', 'num_op',
       'adjust_total_30d', 'log_pre_op_cost', 'numofbrand_post'],
      dtype='object')


In [32]:
lasso_log_cv.coef_

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.51546324,  1.24765562,  0.10076311, -0.01208398, -0.11393066,
         0.        ,  0.        , -0.0121437 ,  0.        , -0.04258878,
         0.        ,  0.        ,  0.        ,  0.        ,  0.11803903,
        -0.10159764,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.05863334,  0.        ,  0.05308079,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.02138832, -0.06043565,
        -0.00194882,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.00252997,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.00894801,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.15203706,  0. 

In [33]:
selected_cols = [ 'num_gpi6_post', 'adjust_total_30d_post', 'generic_rate_post',
       'post_ip_flag', 'post_er_flag', 'post_rx_cost', 'post_total_cost',
       'regionn', 'idx_prodtypen', 'chronic_pain_fibro', 'copd', 'hepatitis',
       'hiv_aids', 'hypertension', 'smoking', 'pre_ip_cost', 'num_op',
       'adjust_total_30d', 'log_pre_op_cost', 'numofbrand_post']

In [34]:
AD_col2 = AD[selected_cols]

### Model Training after feature selection using logistic lasso regression

In [49]:
from xgboost import XGBClassifier
classifier1 = XGBClassifier(use_label_encoder=False, eval_metric='error')

In [35]:
y3 = AD['pdc_80_flag']

In [36]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(AD_col2, y3, test_size=0.2, random_state=42)

In [37]:
# get the best parameters using  cross validation
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
   'max_depth': 5,
    'n_jobs': -1  # Use all available cores
}

In [38]:
# Create DMatrix from training data
dtrain2 = xgb.DMatrix(X_train3, label=y_train3)

cv_results = xgb.cv(
    params = params,
    dtrain = dtrain2,
    nfold = 5,
    num_boost_round = 10,
    verbose_eval = 2
)

[0]	train-logloss:0.65773+0.00049	test-logloss:0.66623+0.00229
[2]	train-logloss:0.60264+0.00104	test-logloss:0.62583+0.00282
[4]	train-logloss:0.56199+0.00186	test-logloss:0.59703+0.00403
[6]	train-logloss:0.53018+0.00256	test-logloss:0.57820+0.00688
[8]	train-logloss:0.50524+0.00330	test-logloss:0.56578+0.00997
[9]	train-logloss:0.49341+0.00359	test-logloss:0.56004+0.01010


In [39]:
best_logloss = cv_results['test-logloss-mean'].min()
best_num_boost_round = cv_results['test-logloss-mean'].idxmin() + 1

In [41]:
best_params = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.1,
    'eval_metric': 'logloss'
}

In [42]:
model2 = xgb.train(
    params = best_params, 
    dtrain = dtrain2, 
    num_boost_round = best_num_boost_round)
dtest3 = xgb.DMatrix(X_test3)
y_pred_prob = model2.predict(dtest3)

# Model2 Evaluation

In [43]:
y_pred3 = np.where(y_pred_prob > 0.5, 1, 0)

In [46]:
# Model Evaluation
print(classification_report(y_test3, y_pred3))
AUC = roc_auc_score(y_test3, y_pred3)
print(f'AUC score: {AUC}')

             precision    recall  f1-score   support

          0       0.78      0.77      0.78       213
          1       0.64      0.65      0.65       132

avg / total       0.73      0.73      0.73       345

AUC score: 0.7130815194195476


In [47]:
X_train3.shape, y_train3.shape

((1377, 20), (1377,))

In [50]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

f1_scorer = make_scorer(f1_score)
# classifier1 is the machine learning model

# Perform 5-fold cross-validation with F1 score as the scoring metric
f1_scores = cross_val_score(classifier1, X_train3, y_train3, cv=5, scoring=f1_scorer)

# Calculate the mean F1 score across all folds
mean_f1_score = f1_scores.mean()
print("Mean F1 Score:", mean_f1_score)


Mean F1 Score: 0.5989207628795798


In [51]:
y_test3.value_counts()

0    213
1    132
Name: pdc_80_flag, dtype: int64